In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import warnings

In [22]:
house_price_train_final = pd.read_csv('Data/house_price_train_final.csv')
house_price_train_final.head()

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,GarageFinish_Unf,KitchenQual_TA,ExterQual_TA
0,7,1710,2,548,856,856,2,8,2003,2003,False,False,False
1,6,1262,2,460,1262,1262,2,6,1976,1976,False,True,True
2,7,1786,2,608,920,920,2,6,2001,2002,False,False,False
3,7,1717,3,642,756,961,1,7,1915,1970,True,False,True
4,8,2198,3,836,1145,1145,2,9,2000,2000,False,False,False


In [23]:
X = house_price_train_final.drop('SalePrice', axis=1)
y = house_price_train_final['SalePrice']

KeyError: "['SalePrice'] not found in axis"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(random_state=42),
    'Lasso Regression': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}
results = {}



In [ ]:
# Evaluar cada modelo con validación cruzada
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    results[name] = {
        'RMSE CV': rmse_scores.mean(),
        'RMSE Std': rmse_scores.std()
    }
    print(f"{name}: RMSE = {rmse_scores.mean():.2f} (±{rmse_scores.std():.2f})")


In [24]:

# Encontrar el mejor modelo basado en RMSE
best_model_name = min(results, key=lambda x: results[x]['RMSE CV'])
print(f"\nEl mejor modelo es: {best_model_name}")


El mejor modelo es: Gradient Boosting


In [25]:

# Optimizar hiperparámetros del mejor modelo
if best_model_name == 'Linear Regression':
    best_model = LinearRegression()
    param_grid = {}
elif best_model_name == 'Ridge Regression':
    best_model = Ridge(random_state=42)
    param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
elif best_model_name == 'Lasso Regression':
    best_model = Lasso(random_state=42)
    param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]}
elif best_model_name == 'ElasticNet':
    best_model = ElasticNet(random_state=42)
    param_grid = {
        'alpha': [0.001, 0.01, 0.1, 1.0],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
    }
elif best_model_name == 'Random Forest':
    best_model = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }
else:  # Gradient Boosting
    best_model = GradientBoostingRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7]
    }

In [26]:

# Realizar búsqueda de hiperparámetros si hay parámetros para optimizar
if param_grid:
    print(f"\nOptimizando hiperparámetros para {best_model_name}...")
    grid_search = GridSearchCV(best_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    best_params = grid_search.best_params_
    print(f"Mejores parámetros: {best_params}")
    final_model = grid_search.best_estimator_
else:
    print(f"\nEntrenando modelo final {best_model_name}...")
    best_model.fit(X_train_scaled, y_train)
    final_model = best_model


Optimizando hiperparámetros para Gradient Boosting...
Mejores parámetros: {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 100}


In [27]:

# Evaluar el modelo final
y_pred = final_model.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\nResultados del modelo final ({best_model_name}):")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")



Resultados del modelo final (Gradient Boosting):
RMSE: 15576.94
R²: -0.0041


In [28]:

# Visualizar residuos
residuos = y_test - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuos, alpha=0.5)
plt.hlines(y=0, xmin=y_pred.min(), xmax=y_pred.max(), colors='r', linestyles='--')
plt.xlabel('Predicciones')
plt.ylabel('Residuos')
plt.title('Residuos vs Predicciones')
plt.savefig('residuos.png')
plt.close()

In [29]:
plt.figure(figsize=(10, 6))
sns.histplot(residuos, kde=True)
plt.xlabel('Residuos')
plt.title('Distribución de residuos')
plt.savefig('distribucion_residuos.png')
plt.close()

In [30]:

# Si el mejor modelo permite análisis de importancia de características
if best_model_name in ['Random Forest', 'Gradient Boosting']:
    # Obtener importancia de características
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': final_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # Visualizar las 15 características más importantes
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(15))
    plt.title('15 características más importantes')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()
    
    print("\nCaracterísticas más importantes:")
    print(feature_importance.head(10))


Características más importantes:
  Feature  Importance
0      Id         1.0


In [31]:

# Cargar datos de prueba y hacer predicciones para enviar
try:
    # Cargar datos de prueba
    house_price_test = pd.read_csv('Data/test.csv')
    
    # Preparar los datos de prueba de la misma manera que los de entrenamiento
    # Esto debe incluir el mismo preprocesamiento aplicado al conjunto de entrenamiento
    # Por simplicidad, asumimos que los datos de prueba ya están procesados 
    # de la misma manera o que las columnas requeridas ya están disponibles
    
    # Asegurarse de que las columnas coincidan con las usadas en el entrenamiento
    missing_cols = set(X.columns) - set(house_price_test.columns)
    for col in missing_cols:
        house_price_test[col] = 0  # Valor predeterminado para columnas faltantes
    
    # Asegurarse de que el orden de las columnas sea el mismo
    house_price_test = house_price_test[X.columns]
    
    # Escalar los datos
    house_price_test_scaled = scaler.transform(house_price_test)
    
    # Hacer predicciones
    test_predictions = final_model.predict(house_price_test_scaled)
    
    # Crear el formato de envío
    submission = pd.DataFrame({
        'Id': house_price_test['Id'] if 'Id' in house_price_test.columns else range(1, len(test_predictions) + 1),
        'SalePrice': test_predictions
    })
    
    # Guardar el archivo de envío
    submission.to_csv('submission.csv', index=False)
    print("\nArchivo de envío creado como 'submission.csv'")
    
except Exception as e:
    print(f"\nNo se pudieron generar predicciones para el conjunto de prueba: {e}")
    print("Nota: Para generar predicciones, asegúrate de que los datos de prueba estén preprocesados correctamente.")


Archivo de envío creado como 'submission.csv'
